In [1]:
# Import numpy for random number generation
import numpy as np
from random import randbytes
# importing Qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
# Import basic plotting tools
from helpers import *
from qiskit.visualization import plot_histogram
from dotenv import load_dotenv
import os

load_dotenv()
provider = QiskitRuntimeService(token=os.environ["ibm_token"], channel="ibm_quantum")

# Selecting a backend
real_backend = provider.backend("ibm_brisbane")
backend = AerSimulator().from_backend(real_backend)

In [10]:
n = 8  # for a local backend n can go as up as 23, after that it raises a Memory Error
qr = QuantumRegister(n, name='qr')
cr = ClassicalRegister(n, name='cr')
Alice = User(
    name = "Alice",
    key = "",
    basis = [],
    qc = QuantumCircuit(qr, cr, name="a"),
    qr = qr,
    cr = cr,
    n = n,
)
Bob = User(
    name = "Bob",
    key = "",
    basis = [],
    qc = QuantumCircuit(qr, cr, name="b"),
    qr = qr,
    cr = cr,
    n = n,
)

In [11]:
Alice.key = int.from_bytes(randbytes(int(n/8)))
# Cast key to binary for encoding
# range: key[0]-key[15] with key[15] least significant figure
Alice.key = np.binary_repr(Alice.key, n) # n is the width
print(Alice.key)
# Encode key as alice qubits 
# IBM's qubits are all set to |0> initially
for index, digit in enumerate(Alice.key):
    if digit == '1':
        Alice.qc.x(Alice.qr[index]) # if key has a '1', change state to |1>
create_basis(Alice)
create_basis(Bob)

01110110


In [15]:
alice_bob = send_state(Alice, Bob)
counts = execute(alice_bob, backend, shots=1000)

#(a,b) = get_shared_key(Alice, Bob, simp = False)
success = 0
keys = list(counts)
print(keys)
for key in keys:
    Bob.key = key[::-1]
    (a, b, s) = get_shared_key(Alice, Bob, simp = True)
    if s:
        success += 1
print(f"Succ rate {success}/{len(keys)}: {success/len(keys)}")

['01100100', '11100101', '01000110', '01110110', '11000010', '01001100', '01100110', '11011110', '11001101', '11100110', '11000100', '01001110', '01000010', '11001100', '01101110', '01101100', '00101110', '11100000', '11011100', '11010000', '00100110', '11101100', '11101101', '11000110', '01001111', '11000111', '11010110', '11101110', '10101110', '11100111', '01000100', '11001110', '01010110', '10000110', '01101000', '01100101', '11111110', '01110100', '11000000', '01000111', '11100100', '01100010', '01111110', '00000110', '01000000', '01001101', '11000101', '11111100', '01100011', '10101100', '11110100', '01101101', '01001000', '11110110', '01011100', '10000100', '10100100', '11100010', '10001100', '01010100', '01100111', '11101010', '01100000', '01101010']
Succ rate 16/64: 0.25
